In [1]:
# Librerias
from datetime import date, datetime
from sqlalchemy import create_engine
import re
import os
import smtplib
import time
import configparser
# Manipulacion y tratamiento de datos
import numpy as np
import pandas as pd

# Graficos
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import hvplot.pandas
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import plotly.express as px
import plotly.graph_objects as go

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams["figure.figsize"] = (12, 8)
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
# Modelacion Arima

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# Auto Arima

from pmdarima import auto_arima

# Metrica de evaluacion
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn import metrics

In [4]:
# Evaluaciones de Modelos

def evaluacion_metricas(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true-y_pred)/y_true)) *100
    print('Evaluación de las metricas resultados:-')
    print(f'MSE es: {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE es: {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE es: {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE es: {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R^2 es: {metrics.r2_score(y_true, y_pred)}', end = '\n\n')

# Modelo para los refunds

In [5]:
# Leer tus datos
df = pd.read_excel('Datos de refunds y ventas.xlsx', sheet_name='Refunds')
df.head()

,Fecha,Amazon,eBay,Shopify,Google,Walmart,Total
0,2020-01-01,421.27,0,0,0.0,0,421.27
1,2020-01-02,5546.81,0,0,0.0,0,5546.81
2,2020-01-03,7327.57,0,0,0.0,0,7327.57
3,2020-01-04,2726.78,0,0,0.0,0,2726.78
4,2020-01-05,1101.16,0,0,0.0,0,1101.16


In [6]:
df.tail()

,Fecha,Amazon,eBay,Shopify,Google,Walmart,Total
1897,2025-03-12,9527.37,143.21,177.18,NaN,0,9847.76
1898,2025-03-13,6497.54,96.64,1015.46,NaN,0,7609.64
1899,2025-03-14,8011.85,94.23,332.28,NaN,0,8438.36
1900,2025-03-15,4081.95,90.69,60.28,NaN,0,4232.92
1901,2025-03-16,1206.46,-,-,NaN,0,1206.46


In [7]:
df = df[['Fecha', 'Total']]
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.set_index('Fecha')

In [8]:
def create_feature(df):
    df = df.copy()  

    df['Año'] = df.index.year
    df['Mes'] = df.index.month
    df['Dia'] = df.index.day
    df['Dia_de_la_Semana'] = df.index.dayofweek
    df['Trimestre'] = df.index.quarter
    df['semana_del_año'] = df.index.isocalendar().week
    df['Dia_del_año'] = df.index.dayofyear
    return df
df= create_feature(df)

In [9]:
def add_lags(df):
    target_map = df.Total.to_dict()
    df['lag1'] = (df.index - pd.Timedelta('32 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('64 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('96 days')).map(target_map)
    return df

df = add_lags(df)

In [10]:
df['lag1'].fillna(0, inplace=True)
df['lag2'].fillna(0, inplace=True)
df['lag3'].fillna(0, inplace=True)

In [11]:
Feature = ['Año', 'Mes', 'Dia', 'Dia_de_la_Semana', 'Trimestre',
       'semana_del_año', 'Dia_del_año','lag1','lag2','lag3']
Target = 'Total'

train_size = len(df) 


train_datos = df.iloc[0:train_size]

print(train_datos.shape)


(1902, 11)


In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train_datos[Feature])

# Crear un nuevo MinMaxScaler para la característica 'Total'
scaler_total = MinMaxScaler(feature_range=(0, 1))
scaled_total = scaler_total.fit_transform(train_datos[['Total']])
                                          
# Crear la red LSTM
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(scaled_train.shape[1], 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
x_train = scaled_train.reshape(scaled_train.shape[0], scaled_train.shape[1], 1)
y_train = scaled_total
model.fit(x_train, y_train, epochs=25, verbose=1)

Epoch 1/25
60/60 [==============================] - 1s 6ms/step - loss: 0.0252
Epoch 2/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0206
Epoch 3/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0202
Epoch 4/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0199
Epoch 5/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0203
Epoch 6/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0191
Epoch 7/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0184
Epoch 8/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0173
Epoch 9/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0158
Epoch 10/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0152
Epoch 11/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0140
Epoch 12/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0131
Epoch 13/25
60/60 [==============================

#### Cambio de Fechas

In [13]:
future = pd.date_range('2025-03-17','2025-04-06', freq='1D')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_future = pd.concat([df, future_df])
df_and_future = create_feature(df_and_future)
df_and_future = add_lags(df_and_future)

In [14]:
future_w_feature =df_and_future.query('isFuture').copy()

In [15]:
# Preparar los datos futuros
future_w_feature_scaled = scaler.transform(future_w_feature[Feature])

# Hacer las predicciones
x_future = future_w_feature_scaled.reshape(future_w_feature_scaled.shape[0], future_w_feature_scaled.shape[1], 1)
predictions = model.predict(x_future)

# Desescalar las predicciones
predictions = scaler_total.inverse_transform(predictions)


1/1 [==============================] - 0s 121ms/step


In [16]:
predictions

array([[ 8516.41   ],
       [13659.311  ],
       [10628.294  ],
       [ 7413.479  ],
       [ 6899.3213 ],
       [ 3064.353  ],
       [   82.06111],
       [ 7379.3887 ],
       [12249.649  ],
       [ 9959.34   ],
       [ 7074.519  ],
       [ 6755.3877 ],
       [ 2926.1736 ],
       [  183.26514],
       [ 8712.549  ],
       [15831.112  ],
       [13046.857  ],
       [11084.327  ],
       [12020.466  ],
       [ 8310.287  ],
       [ 4627.273  ]], dtype=float32)

In [17]:
future_w_feature['refunds_pred'] = predictions

In [18]:
future_w_feature

,Total,Año,Mes,Dia,Dia_de_la_Semana,Trimestre,semana_del_año,Dia_del_año,lag1,lag2,lag3,isFuture,refunds_pred
2025-03-17,NaN,2025,3,17,0,1,12,76,9041.27,1497.84,13364.26,True,8516.410156
2025-03-18,NaN,2025,3,18,1,1,12,77,10547.39,11369.99,13582.08,True,13659.310547
2025-03-19,NaN,2025,3,19,2,1,12,78,7194.61,21005.00,15726.08,True,10628.293945
2025-03-20,NaN,2025,3,20,3,1,12,79,1489.73,12656.05,7200.37,True,7413.479004
2025-03-21,NaN,2025,3,21,4,1,12,80,5751.11,12981.16,1979.95,True,6899.321289
2025-03-22,NaN,2025,3,22,5,1,12,81,14277.11,12269.48,12147.85,True,3064.353027
2025-03-23,NaN,2025,3,23,6,1,12,82,11181.71,5296.62,20576.61,True,82.061111
2025-03-24,NaN,2025,3,24,0,1,13,83,9437.69,2093.81,17586.00,True,7379.388672
2025-03-25,NaN,2025,3,25,1,1,13,84,10602.01,5840.43,13297.01,True,12249.649414
2025-03-26,NaN,2025,3,26,2,1,13,85,5253.78,21311.69,10199.61,True,9959.339844


In [19]:
fig = px.line(future_w_feature, x=future_w_feature.index, y ='refunds_pred', template='plotly_dark', title='Predicciones con el modelo de Redes Neuronales')
fig.show()

# Modelo de Ventas

In [20]:
ventas = pd.read_excel('Datos de refunds y ventas.xlsx', sheet_name='Ventas')
ventas.head(15)

,Fecha,Total
0,2020-01-01,114838.8201
1,2020-01-02,132846.9601
2,2020-01-03,130644.7101
3,2020-01-04,120840.9203
4,2020-01-05,129750.7500
5,2020-01-06,142831.5700
6,2020-01-07,140678.2699
7,2020-01-08,130589.3798
8,2020-01-09,114432.2300
9,2020-01-10,109847.7200


In [21]:
ventas = ventas[['Fecha', 'Total']]
ventas['Fecha'] = pd.to_datetime(ventas['Fecha'])
ventas = ventas.set_index('Fecha')

In [22]:
ventas= create_feature(ventas)
ventas = add_lags(ventas)

ventas['lag1'].fillna(0, inplace=True)
ventas['lag2'].fillna(0, inplace=True)
ventas['lag3'].fillna(0, inplace=True)

In [23]:
Feature = ['Año', 'Mes', 'Dia', 'Dia_de_la_Semana', 'Trimestre',
       'semana_del_año', 'Dia_del_año','lag1','lag2','lag3']
Target = 'Total'

train_size = len(ventas) 


train_datos = ventas.iloc[0:train_size]

print(train_datos.shape)

(1902, 11)


In [24]:
scaler_sales = MinMaxScaler()
scaled_train_sales = scaler_sales.fit_transform(train_datos[Feature])

# Crear un nuevo MinMaxScaler para la característica 'Total'
scaler_total_sales = MinMaxScaler()
scaled_total_sales = scaler_total_sales.fit_transform(train_datos[['Total']])
                                          
# Crear la red LSTM
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(scaled_train_sales.shape[1], 1)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
x_train = scaled_train_sales.reshape(scaled_train_sales.shape[0], scaled_train_sales.shape[1], 1)
y_train = scaled_total_sales
model.fit(x_train, y_train, epochs=25, verbose=1)

Epoch 1/25
60/60 [==============================] - 1s 6ms/step - loss: 0.0273
Epoch 2/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0111
Epoch 3/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0103
Epoch 4/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0102
Epoch 5/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0099
Epoch 6/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0098
Epoch 7/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0096
Epoch 8/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0092
Epoch 9/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0087
Epoch 10/25
60/60 [==============================] - 0s 7ms/step - loss: 0.0088
Epoch 11/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0081
Epoch 12/25
60/60 [==============================] - 0s 6ms/step - loss: 0.0081
Epoch 13/25
60/60 [==============================

#### Cambio de Fechas

In [25]:
future_sales = pd.date_range('2025-03-17','2025-04-06', freq='1D')
future_df_sales = pd.DataFrame(index=future_sales)
future_df_sales['isFuture'] = True  

ventas['isFuture'] = False
df_and_future_sales = pd.concat([ventas, future_df_sales])
df_and_future_sales = create_feature(df_and_future_sales)
df_and_future_sales = add_lags(df_and_future_sales)


In [26]:
future_w_feature_2 =df_and_future_sales.query('isFuture').copy()

In [27]:
# Preparar los datos futuros
future_w_feature_scaled_2 = scaler_sales.transform(future_w_feature_2[Feature])

# Hacer las predicciones
x_future_2 = future_w_feature_scaled_2.reshape(future_w_feature_scaled_2.shape[0], future_w_feature_scaled_2.shape[1], 1)
predictions_2 = model.predict(x_future_2)

# Desescalar las predicciones
predictions_2 = scaler_total_sales.inverse_transform(predictions_2)

1/1 [==============================] - 0s 109ms/step


In [28]:
future_w_feature_2['sales_pred'] = predictions_2 

In [29]:
predictions_2

array([[218086.5 ],
       [225150.97],
       [216356.5 ],
       [214127.34],
       [204836.38],
       [189465.97],
       [171651.31],
       [208739.22],
       [214166.86],
       [203291.66],
       [202266.6 ],
       [199624.5 ],
       [188164.66],
       [171989.67],
       [208305.25],
       [253508.05],
       [246151.42],
       [237385.97],
       [234789.56],
       [227561.45],
       [208487.28]], dtype=float32)

In [30]:
future_w_feature_2

,Total,Año,Mes,Dia,Dia_de_la_Semana,Trimestre,semana_del_año,Dia_del_año,lag1,lag2,lag3,isFuture,sales_pred
2025-03-17,NaN,2025,3,17,0,1,12,76,213816.5098,171835.0103,262737.1401,True,218086.500000
2025-03-18,NaN,2025,3,18,1,1,12,77,180507.6600,269371.0292,280403.9296,True,225150.968750
2025-03-19,NaN,2025,3,19,2,1,12,78,150857.0205,284339.3596,249646.2899,True,216356.500000
2025-03-20,NaN,2025,3,20,3,1,12,79,176482.2900,263559.9301,278750.3100,True,214127.343750
2025-03-21,NaN,2025,3,21,4,1,12,80,265286.8599,236801.6300,232909.7000,True,204836.375000
2025-03-22,NaN,2025,3,22,5,1,12,81,310632.1690,190402.1700,303931.5199,True,189465.968750
2025-03-23,NaN,2025,3,23,6,1,12,82,300421.7803,134986.1799,241176.3101,True,171651.312500
2025-03-24,NaN,2025,3,24,0,1,13,83,278763.4900,158491.0000,250857.3797,True,208739.218750
2025-03-25,NaN,2025,3,25,1,1,13,84,216318.5102,228262.0000,205817.4801,True,214166.859375
2025-03-26,NaN,2025,3,26,2,1,13,85,161158.2300,221234.8400,170830.2900,True,203291.656250


### Extraccion de las predicciones

In [31]:
future_w_feature['Sales_Pred'] = future_w_feature_2['sales_pred']

predicciones_redneuronal = pd.DataFrame(index=future_w_feature.index)
predicciones_redneuronal['Refunds'] = future_w_feature['refunds_pred']
predicciones_redneuronal['Sales'] = future_w_feature['Sales_Pred']
print(predicciones_redneuronal.head())
predicciones_redneuronal.to_excel('predicciones red neuronal.xlsx', index=True)

                 Refunds         Sales
2025-03-17   8516.410156  218086.50000
2025-03-18  13659.310547  225150.96875
2025-03-19  10628.293945  216356.50000
2025-03-20   7413.479004  214127.34375
2025-03-21   6899.321289  204836.37500


In [32]:
import plotly.express as px


historia_refunds = df.tail(31)

fig_refunds = px.line(historia_refunds, x=historia_refunds.index, y='Total', template='plotly_dark', title='Historial de Refunds y Predicciones')
fig_refunds.add_trace(go.Scatter(x=future_w_feature.index, y=future_w_feature['refunds_pred'], mode='lines', name='Refunds Predicciones'))

fig_refunds.update_layout(xaxis_title='Fecha', yaxis_title='Refunds', legend_title='Leyenda')

# Mostrar el gráfico
fig_refunds.show()



historia_ventas = ventas.tail(31)

# Crear gráfico para ventas con datos históricos
fig_ventas = px.line(historia_ventas, x=historia_ventas.index, y='Total', template='plotly_dark', title='Historial de Ventas y Predicciones')
fig_ventas.add_trace(go.Scatter(x=future_w_feature.index, y=future_w_feature['Sales_Pred'], mode='lines', name='Ventas Predicciones'))

# Personalizar diseño
fig_ventas.update_layout(xaxis_title='Fecha', yaxis_title='Ventas', legend_title='Leyenda')

# Mostrar el gráfico
fig_ventas.show()



